In [49]:
#!pip install pandas
#pip install missingno
#!pip install dash
#!pip install streamlit
#pip install --upgrade notebook jupyter nbformat
#pip install nbformat --upgrade
#!pip install plotly

In [50]:
import pandas as pd
import streamlit as st
import pandas as pd
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
import missingno

In [51]:
covid19=pd.read_csv('covid_19_clean_complete.csv')
covid19.head()

,Province/State,Country/Region,Lat,Long,Date,Confirmed,Deaths,Recovered,Active,WHO Region
0,NaN,Afghanistan,33.93911,67.709953,2020-01-22,0,0,0,0,Eastern Mediterranean
1,NaN,Albania,41.15330,20.168300,2020-01-22,0,0,0,0,Europe
2,NaN,Algeria,28.03390,1.659600,2020-01-22,0,0,0,0,Africa
3,NaN,Andorra,42.50630,1.521800,2020-01-22,0,0,0,0,Europe
4,NaN,Angola,-11.20270,17.873900,2020-01-22,0,0,0,0,Africa


In [52]:
usa=pd.read_csv('usa_county_wise.csv')
usa.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,Combined_Key,Date,Confirmed,Deaths
0,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,"American Samoa, US",1/22/20,0,0
1,316,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,"Guam, US",1/22/20,0,0
2,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,"Northern Mariana Islands, US",1/22/20,0,0
3,63072001,PR,PRI,630,72001.0,Adjuntas,Puerto Rico,US,18.180117,-66.754367,"Adjuntas, Puerto Rico, US",1/22/20,0,0
4,63072003,PR,PRI,630,72003.0,Aguada,Puerto Rico,US,18.360255,-67.175131,"Aguada, Puerto Rico, US",1/22/20,0,0


In [53]:
print(covid19.columns)
print(usa.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', 'Date', 'Confirmed',
       'Deaths', 'Recovered', 'Active', 'WHO Region'],
      dtype='object')
Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Province_State',
       'Country_Region', 'Lat', 'Long_', 'Combined_Key', 'Date', 'Confirmed',
       'Deaths'],
      dtype='object')


In [54]:
missing_values_covid19 = covid19['WHO Region'].isnull().sum()
missing_values_covid19 

0

In [55]:
missing_values_usa= usa['Deaths'].isnull().sum()
missing_values_usa 

0

In [56]:
print(covid19[['Province/State', 'Confirmed', 'Deaths', 'Recovered']].head())

# حساب عدد الحالات المؤكدة لكل مقاطعة/ولاية
cases_by_province = covid19.groupby('Province/State').agg({
    'Confirmed': 'sum',
    'Deaths': 'sum',
    'Recovered': 'sum'
}).reset_index()

print(cases_by_province)

# عرض المقاطعات/الولايات التي تحتوي على أكبر عدد من الحالات المؤكدة
top_provinces = cases_by_province.sort_values(by='Confirmed', ascending=False).head(10)
print(top_provinces)

  Province/State  Confirmed  Deaths  Recovered
0            NaN          0       0          0
1            NaN          0       0          0
2            NaN          0       0          0
3            NaN          0       0          0
4            NaN          0       0          0
                  Province/State  Confirmed  Deaths  Recovered
0                        Alberta     751219   14245          0
1                       Anguilla        361       0        290
2                          Anhui     172497    1007     155768
3                          Aruba      12090     289       9629
4   Australian Capital Territory      13174     339      11264
..                           ...        ...     ...        ...
73             Western Australia      70669    1005      59833
74                      Xinjiang      14160     474      11280
75                         Yukon       1276       0          0
76                        Yunnan      32118     319      28400
77                      Z

In [87]:
# عرض أسماء الأعمدة وأنواع البيانات
print(covid19.dtypes)

# اختيار الأعمدة العددية فقط
numeric_data = covid19.select_dtypes(include=['number'])
print(numeric_data.columns)

Province/State     object
Country/Region     object
Lat               float64
Long              float64
Date               object
Confirmed           int64
Deaths              int64
Recovered           int64
Active              int64
WHO Region         object
dtype: object
Index(['Lat', 'Long', 'Confirmed', 'Deaths', 'Recovered', 'Active'], dtype='object')


In [88]:

# استبدال القيم الفارغة في عمود 'Province/State' بقيمة 'Unknown'
covid19['Province/State'] = covid19['Province/State'].fillna('Unknown')

# عرض البيانات للتأكد من التعديل
print(covid19.head())

  Province/State Country/Region       Lat       Long        Date  Confirmed  \
0        Unknown    Afghanistan  33.93911  67.709953  2020-01-22          0   
1        Unknown        Albania  41.15330  20.168300  2020-01-22          0   
2        Unknown        Algeria  28.03390   1.659600  2020-01-22          0   
3        Unknown        Andorra  42.50630   1.521800  2020-01-22          0   
4        Unknown         Angola -11.20270  17.873900  2020-01-22          0   

   Deaths  Recovered  Active             WHO Region  
0       0          0       0  Eastern Mediterranean  
1       0          0       0                 Europe  
2       0          0       0                 Africa  
3       0          0       0                 Europe  
4       0          0       0                 Africa  


In [79]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import numpy as np



covid19=pd.read_csv('covid_19_clean_complete.csv')


# إعداد بيانات افتراضية
np.random.seed(42)
dates = pd.date_range(start="2024-01-01", periods=30, freq='D')

data = pd.DataFrame({
    'Province/State': np.random.choice(['Province1', 'Province2', 'Province3', 'Province4', 'Province5'], size=30),
    'Confirmed': np.random.randint(500, 2000, size=30),
    'Deaths': np.random.randint(10, 100, size=30),
    'Recovered': np.random.randint(200, 1500, size=30),
    'Date': dates,
})

# بيانات إضافية
category_data = pd.DataFrame({
    'Category': ['Confirmed', 'Deaths', 'Recovered'],
    'Value': [data['Confirmed'].sum(), data['Deaths'].sum(), data['Recovered'].sum()]
})

sunburst_data = pd.DataFrame({
    'labels': ['Total Cases', 'Confirmed', 'Deaths', 'Recovered'],
    'parents': ['', 'Total Cases', 'Total Cases', 'Total Cases'],
    'values': [data['Confirmed'].sum() + data['Deaths'].sum() + data['Recovered'].sum(),
               data['Confirmed'].sum(), data['Deaths'].sum(), data['Recovered'].sum()]
})

# إنشاء الرسوم البيانية
# مخطط مساحي
area_chart = px.area(data, x='Date', y='Confirmed', title='Area Chart of Confirmed Cases Over Time')

# مخطط عمودي مكدس
stacked_column_chart = px.bar(data, x='Province/State', y=['Confirmed', 'Deaths', 'Recovered'],
                              title='Stacked Column Chart of COVID-19 Cases')

# مخطط شريطي أفقي
horizontal_bar_chart = px.bar(data, y='Province/State', x='Confirmed', orientation='h',
                              title='Horizontal Bar Chart of Confirmed Cases by Province/State')

# مخطط شجري
tree_map_chart = px.treemap(category_data, path=['Category'], values='Value',
                           title='Tree Map of COVID-19 Case Distribution')

# مخطط تدفق
sunburst_chart = px.sunburst(sunburst_data, names='labels', parents='parents', values='values',
                             title='Sunburst Chart of COVID-19 Cases')

# مخطط سطح
x = np.linspace(-5, 5, 50)
y = np.linspace(-5, 5, 50)
X, Y = np.meshgrid(x, y)
Z = np.sin(np.sqrt(X**2 + Y**2))  # تجنب المصفوفات العشوائية

surface_chart = go.Figure(data=[go.Surface(z=Z, x=x, y=y)])
surface_chart.update_layout(title='Surface Plot of Sine Function')

# مخطط بولينغ
box_plot = px.box(data, y='Confirmed', title='Box Plot of Confirmed Cases')

# إنشاء تطبيق Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("COVID-19 Dashboard"),

    html.Div([
        html.H2("Area Chart of Confirmed Cases Over Time"),
        dcc.Graph(
            id='area-chart',
            figure=area_chart
        )
    ]),

    html.Div([
        html.H2("Stacked Column Chart of COVID-19 Cases"),
        dcc.Graph(
            id='stacked-column-chart',
            figure=stacked_column_chart
        )
    ]),

    html.Div([
        html.H2("Horizontal Bar Chart of Confirmed Cases"),
        dcc.Graph(
            id='horizontal-bar-chart',
            figure=horizontal_bar_chart
        )
    ]),

    html.Div([
        html.H2("Tree Map of COVID-19 Case Distribution"),
        dcc.Graph(
            id='tree-map-chart',
            figure=tree_map_chart
        )
    ]),

    html.Div([
        html.H2("Sunburst Chart of COVID-19 Cases"),
        dcc.Graph(
            id='sunburst-chart',
            figure=sunburst_chart
        )
    ]),

    html.Div([
        html.H2("Surface Plot of Sine Function"),
        dcc.Graph(
            id='surface-chart',
            figure=surface_chart
        )
    ]),

    html.Div([
        html.H2("Box Plot of Confirmed Cases"),
        dcc.Graph(
            id='box-plot',
            figure=box_plot
        )
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [80]:
import dash
from dash import dcc, html
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go



usa=pd.read_csv('usa_county_wise.csv')


# إعداد بيانات افتراضية
data = pd.DataFrame({
    'Province/State': ['Province1', 'Province2', 'Province3', 'Province4', 'Province5'],
    'Confirmed': [1000, 900, 800, 700, 600],
    'Deaths': [50, 45, 40, 35, 30],
    'Recovered': [800, 700, 600, 500, 400],
    'Date': pd.date_range(start="2024-01-01", periods=5, freq='D'),
    'Category': ['Confirmed', 'Deaths', 'Recovered', 'Confirmed', 'Deaths'],
})

# إعداد بيانات للمخططات
time_series_data = pd.DataFrame({
    'Date': pd.date_range(start="2024-01-01", periods=10, freq='D'),
    'Confirmed': [1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700, 1800, 1900],
})

pie_data = pd.DataFrame({
    'Category': ['Confirmed', 'Deaths', 'Recovered'],
    'Count': [5000, 300, 2000]
})

# مخطط شريطي
bar_chart = px.bar(data, x='Province/State', y='Confirmed', title='Confirmed Cases by Province/State')

# مخطط شريطي مكدس
stacked_bar_chart = px.bar(data, x='Province/State', y=['Confirmed', 'Deaths', 'Recovered'], title='Stacked Bar Chart of COVID-19 Cases')

# مخطط خطي
line_chart = px.line(time_series_data, x='Date', y='Confirmed', title='Daily Confirmed Cases')

# مخطط نقطي
scatter_chart = px.scatter(data, x='Confirmed', y='Deaths', text='Province/State', title='Scatter Plot of Confirmed Cases vs Deaths')

# مخطط دائري
pie_chart = px.pie(pie_data, names='Category', values='Count', title='Pie Chart of COVID-19 Case Distribution')

# إنشاء تطبيق Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("COVID-19 Dashboard"),

    html.Div([
        html.H2("Bar Chart of Confirmed Cases"),
        dcc.Graph(
            id='bar-chart',
            figure=bar_chart
        )
    ]),

    html.Div([
        html.H2("Stacked Bar Chart of COVID-19 Cases"),
        dcc.Graph(
            id='stacked-bar-chart',
            figure=stacked_bar_chart
        )
    ]),

    html.Div([
        html.H2("Daily Confirmed Cases"),
        dcc.Graph(
            id='line-chart',
            figure=line_chart
        )
    ]),

    html.Div([
        html.H2("Scatter Plot of Confirmed Cases vs Deaths"),
        dcc.Graph(
            id='scatter-chart',
            figure=scatter_chart
        )
    ]),

    html.Div([
        html.H2("Pie Chart of COVID-19 Case Distribution"),
        dcc.Graph(
            id='pie-chart',
            figure=pie_chart
        )
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)


In [82]:
import dash
from dash import dcc, html, dash_table
import pandas as pd
import plotly.express as px



usa=pd.read_csv('usa_county_wise.csv')


# إعداد بيانات top_provinces
top_provinces = pd.DataFrame({
    'Province/State': ['Province1', 'Province2', 'Province3', 'Province4', 'Province5', 'Province6', 'Province7', 'Province8', 'Province9', 'Province10'],
    'Confirmed': [1000, 900, 800, 700, 600, 500, 400, 300, 200, 100],
    'Deaths': [50, 45, 40, 35, 30, 25, 20, 15, 10, 5],
    'Recovered': [800, 700, 600, 500, 400, 300, 200, 100, 90, 80]
})

# إعداد بيانات زمنية لمخطط تراكمي
date_range = pd.date_range(start="2024-01-01", periods=10, freq='D')
cumulative_data = pd.DataFrame({
    'Date': date_range,
    'Cumulative_Confirmed': [1000, 2200, 3400, 4600, 5800, 7000, 8200, 9400, 10600, 11800]
})

# إعداد بيانات لمخطط دائري
total_confirmed = top_provinces['Confirmed'].sum()
total_deaths = top_provinces['Deaths'].sum()
total_recovered = top_provinces['Recovered'].sum()

pie_data = pd.DataFrame({
    'Category': ['Confirmed', 'Deaths', 'Recovered'],
    'Count': [total_confirmed, total_deaths, total_recovered]
})

# إنشاء تطبيق Dash
app = dash.Dash(__name__)

# مخطط دائري
pie_chart = px.pie(pie_data, names='Category', values='Count',
                   title='COVID-19 Case Distribution',
                   color='Category',
                   color_discrete_map={'Confirmed': 'blue', 'Deaths': 'red', 'Recovered': 'green'})

# مخطط تراكمي
cumulative_chart = px.line(cumulative_data, x='Date', y='Cumulative_Confirmed',
                           title='Cumulative Confirmed Cases Over Time',
                           labels={'Cumulative_Confirmed': 'Cumulative Confirmed Cases'})

app.layout = html.Div([
    html.H1("COVID-19 Dashboard"),
    
    # قسم للمخطط الدائري
    html.Div([
        html.H2("COVID-19 Case Distribution"),
        dcc.Graph(
            id='pie-chart',
            figure=pie_chart
        )
    ]),
    
    # قسم للمخطط التراكمي
    html.Div([
        html.H2("Cumulative Confirmed Cases"),
        dcc.Graph(
            id='cumulative-chart',
            figure=cumulative_chart
        )
    ]),
    
    # قسم للجدول
    html.Div([
        html.H2("Top 10 Provinces/States"),
        dash_table.DataTable(
            id='cases-table',
            columns=[{'name': c, 'id': c} for c in top_provinces.columns],
            data=top_provinces.to_dict('records'),
            style_table={'overflowX': 'auto'},
            style_cell={'textAlign': 'left'},
            style_header={'backgroundColor': 'rgb(230, 230, 230)', 'fontWeight': 'bold'},
            page_size=10
        )
    ])
])

if __name__ == '__main__':
    app.run_server(debug=True)
